<h1 style="text-align: center;"> FINAL PROJECT </h1>
<h2 style="text-align: center;"> Dự đoán điểm IT001 từ dữ liệu làm bài trên WECODE </h2>

---

#### Lớp: CS114.P21

#### Thành viên nhóm

1. Nguyễn Chí Cường - 23520199
2. Hứa Mạnh Tân - 23521396
3. Nguyễn Tấn Tài - 23521376

#### MỤC LỤC

---

## 1. Chuẩn bị và sơ lược về bộ dữ liệu

### 1.1. Import thư viện cần thiết

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1.2. Load bộ dữ liệu

In [13]:
# Đường dẫn đến các tập dữ liệu
WECODE_DATASET = "../dataset/wecode/annonimized.csv"
QT_DATASET = "../dataset/wecode/qt-public.csv"
TH_DATASET = "../dataset/wecode/th-public.csv"
CK_DATASET = "../dataset/wecode/ck-public.csv"

In [14]:
wecode_df = pd.read_csv(WECODE_DATASET).rename(
    columns={
        "concat('it001',`assignment_id`)": "assignment_id",
        "concat('it001',`problem_id`)": "problem_id",
        "concat('it001', username)": "username",
        "concat('it001',`language_id`)": "language_id",
    }
)

qt_df = pd.read_csv(QT_DATASET).rename(columns={"hash": "username", "diemqt": "QT"})
th_df = pd.read_csv(TH_DATASET).rename(columns={"hash": "username"})
ck_df = pd.read_csv(CK_DATASET).rename(columns={"hash": "username"})

### 1.3. Thông tin bộ dữ liệu

In [15]:
qt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  761 non-null    object
 1   QT        755 non-null    object
dtypes: object(2)
memory usage: 12.0+ KB


In [16]:
qt_df.nunique()

username    761
QT           21
dtype: int64

In [17]:
th_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   username  761 non-null    object
 1   TH        755 non-null    object
dtypes: object(2)
memory usage: 12.0+ KB


In [18]:
th_df.nunique()

username    761
TH           21
dtype: int64

In [19]:
ck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   username  761 non-null    object 
 1   CK        755 non-null    float64
dtypes: float64(1), object(1)
memory usage: 12.0+ KB


In [20]:
ck_df.nunique()

username    761
CK           21
dtype: int64

Từ thông tin của 3 dataframe điểm trên, ta có thể thấy những thông tin sau:
- Cả 3 dataframe đều có 761 `username` non-null và khác nhau
- Cả 3 dataframe có cột điểm chỉ có 755/761 giá trị là non-null

In [7]:
print(
    "Number of similar usernames between qt_df and th_df: "
    + str(sum(qt_df["username"] == th_df["username"]))
)
print(
    "Number of similar usernames between qt_df and ck_df: "
    + str(sum(qt_df["username"] == ck_df["username"]))
)

Number of similar usernames between qt_df and th_df: 761
Number of similar usernames between qt_df and ck_df: 761


Nhận thấy cả 3 dataframe chứa điểm quá trình, thực hành, cuối kì của cùng 761 sinh viên, vậy nên ta có thể hợp nhất cả 3 dataframe được load riêng lẻ thành 1 dataframe duy nhất dể tiện hơn trong việc thao tác.

In [8]:
# Hợp nhất điểm từ 3 dataframe thành 1 dataframe score_df duy nhất
score_df = pd.merge(
    pd.merge(qt_df, th_df, on="username", how="inner"),
    ck_df,
    on="username",
    how="inner",
)

## 2. Phân tích dữ liệu

## 3. Tiền xử lí dữ liệu

### 3.1. Xử lí dữ liệu khuyết thiếu

Bây giờ, ta xem xét những hàng bị bỏ trống (NaN) trong `score_df` để thực hiện lọc hoặc lấp đầy chúng, tránh lỗi trong quá trình thao tác.

In [9]:
score_df[(score_df["QT"].isna()) | (score_df["TH"].isna()) | (score_df["CK"].isna())]

,username,QT,TH,CK
38,06024da8147b5cf6105beb4c403f67cf1ff0aea1,NaN,NaN,NaN
83,0d815307d5307124d148bbebfb41497230c4bd15,NaN,NaN,NaN
91,0eb93817d6eb3d5bd3e44dbc75971eea193b924f,NaN,NaN,NaN
195,209d7c77e7a150b6990a3372f7cb24da0c0e7e07,NaN,NaN,NaN
210,2383a9a33025aa8759fee9a890f6420df0fc8b47,NaN,NaN,NaN
244,2855b4b4d03c37c94117e615a25e0e321ec8d77a,NaN,NaN,NaN


Ta dễ dàng nhận thấy rằng chỉ có 6 `username` có ít nhất 1 trong 3 cột bị bỏ trống (NaN), và thực chất là cả 3 cột đều bị bỏ trống. Vì vậy ta có thể loại những `username` này ra khỏi dataframe `score_df`.

In [10]:
# Xử lý lọc dữ liệu không hợp lệ
score_df = score_df.dropna()

Sau khi loại bỏ NaN thì kết quả dataframe `score_df` thu được:

In [11]:
score_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 755 entries, 0 to 760
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   username  755 non-null    object 
 1   QT        755 non-null    object 
 2   TH        755 non-null    object 
 3   CK        755 non-null    float64
dtypes: float64(1), object(3)
memory usage: 29.5+ KB
